In [1]:
from models.architecture.rnn_net import RnnPredictCandles
from tools.dataset import Dataset
from models.processing.create_features import createFeatures
from models.processing.processing_data import Transformer

import numpy as np
import pandas as pd
import plotly.graph_objects as go

In [2]:
dataset = Dataset().get_data(days=90, ticker='BTCUSDT', ts='1h')

In [7]:
features = createFeatures().create_features(dataset[['open', 'high', 'low', 'close']])
cols = features.columns
transformer = Transformer()
X, y, indexes = transformer.transform_to_lstm_input(
    data=features,
    cols=cols,
    window_size=100,
    step_size=0,
    ahead=12
)

0it [00:00, ?it/s]

[X] Ooops!!! It's True, the series 'volume' was not found in open, high, low, close


/home/serg/projects/onramp/quant-ai/models/processing/create_features.py:11: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

39it [00:00, 69.31it/s]


In [8]:
X_split = transformer.split_data(X, 0.1)
y_split = transformer.split_data(y, 0.1)
indexes = transformer.split_data(indexes, 0.1)

model = RnnPredictCandles(
    input_shape=X.shape[1:],
    output_shape=y.shape[1:],
    epochs=500
)
model.verbose = 1
model.batch_size = 128
model.build_model()
model.fit_train_test(X_split[0], y_split[0], X_split[1], y_split[1])


Epoch 1/500
2/2 [==============================] - 6s 2s/step - loss: 0.2294 - accuracy: 0.2334 - val_loss: 0.0587 - val_accuracy: 0.1361
Epoch 2/500
2/2 [==============================] - 1s 430ms/step - loss: 0.1436 - accuracy: 0.2355 - val_loss: 0.0499 - val_accuracy: 0.2050
Epoch 3/500
2/2 [==============================] - 1s 482ms/step - loss: 0.1239 - accuracy: 0.2462 - val_loss: 0.0375 - val_accuracy: 0.2564
Epoch 4/500
2/2 [==============================] - 1s 465ms/step - loss: 0.0988 - accuracy: 0.2564 - val_loss: 0.0270 - val_accuracy: 0.2340
Epoch 5/500
2/2 [==============================] - 1s 479ms/step - loss: 0.0779 - accuracy: 0.2538 - val_loss: 0.0226 - val_accuracy: 0.2091
Epoch 6/500
2/2 [==============================] - 0s 400ms/step - loss: 0.0648 - accuracy: 0.2568 - val_loss: 0.0207 - val_accuracy: 0.1996
Epoch 7/500
2/2 [==============================] - 0s 407ms/step - loss: 0.0552 - accuracy: 0.2453 - val_loss: 0.0191 - val_accuracy: 0.1989
Epoch 8/500
2/2 

In [9]:
predicted = model.predict(X_split[1])
predicted = transformer.inverse_min_max_scaler_y(predicted[-1])
print(f'predicted: {predicted}')

56/56 [==============================] - 1s 10ms/step
predicted: [[19715.967 19990.656 19467.525 19635.617]
 [19486.096 19973.516 19507.432 19739.744]
 [19419.613 19699.055 19142.906 19781.81 ]
 [19522.627 19847.287 19312.758 19523.424]
 [19264.22  19881.06  19221.936 19452.102]
 [19047.38  19601.611 19240.533 19356.807]
 [19392.996 19856.625 19399.451 19436.871]
 [19698.469 19765.432 19177.205 19383.066]
 [19610.242 19869.783 18978.904 19730.422]
 [19464.537 19613.455 19500.447 19559.07 ]
 [19707.746 19631.15  19519.928 19675.234]
 [19337.562 19953.197 19396.127 19740.084]]


# Predict real time for the next 12 Hours (candles)

In [10]:
expected_prices = pd.DataFrame(predicted, columns=['open', 'high', 'low', 'close'])
fig = go.Figure(data=[go.Candlestick(
                x=expected_prices.index,
                open=expected_prices['open'],
                high=expected_prices['high'],
                low=expected_prices['low'],
                close=expected_prices['close'])])
fig.show()

In [8]:
indexes[1]

Timestamp('2022-08-07 02:00:00')

In [9]:
model.verbose

1